In [1]:
#ft_model = "small-trainoutput-nods"
ft_model = "ds-zero1-t5small"
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch import cuda

device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_map)

#model = AutoModelForSeq2SeqLM.from_pretrained(ft_model,device_map=device_map)
model = AutoModelForSeq2SeqLM.from_pretrained(ft_model,device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(ft_model)
from datasets import load_dataset

test_data = load_dataset('wikisql', split='test')

def translate_to_sql(text):
    inputs = tokenizer(text, padding='longest', max_length=128, return_tensors='pt').to("cpu")
    #inputs = tokenizer(text, padding='longest', max_length=128, return_tensors='pt').to(device_map)
    input_ids = inputs.input_ids
    #attention_mask = inputs.attention_mask
    #output = model.generate(input_ids, attention_mask=attention_mask, max_length=64)
    output = model.generate(input_ids, max_length=96)
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

def print_param_precision(model):
  dtypes = {}
  for _, p in model.named_parameters():
      dtype = p.dtype
      if dtype not in dtypes:
          dtypes[dtype] = 0
      dtypes[dtype] += p.numel()
  total = 0
  for k, v in dtypes.items():
      total += v
  for k, v in dtypes.items():
      print(f"{k}, {v / 10**6:.4f} M, {v / total*100:.2f} %")

def print_parameters(model):
  # Count the total parameters
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Total parameters: {total_params/10**6:.4f} M")

print("\nParameters:")
print_parameters(model)
print("\nData types:")
print_param_precision(model)

/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0

Parameters:
Total parameters: 60.5066 M

Data types:
torch.float32, 60.5066 M, 100.00 %


In [9]:
import time
start_time = time.time()

for i in range(10,20,2):
  print('Test Instruction: ' + test_data[i]['question'])
  print('Model Prediction: ' + translate_to_sql('translate to SQL: ' + test_data[i]['question']))
  print('Expected Answer: ' + test_data[i]['sql']['human_readable'])
  print('=================================\n')

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Inference took {elapsed_time:.2f} seconds")
with open("Inference_time.txt", "w") as file:
    file.write(f"Inference took {elapsed_time:.2f} seconds")

Test Instruction: How many different nationalities do the players of New Jersey Devils come from?
Model Prediction: SELECT COUNT Nationalities FROM FROM table WHERE Players = New Jersey Devils
Expected Answer: SELECT COUNT Nationality FROM table WHERE NHL team = New Jersey Devils

Test Instruction: What is the nationality of the player from Vancouver Canucks?
Model Prediction: SELECT Nationality FROM table WHERE Player = Vancouver Canucks
Expected Answer: SELECT Nationality FROM table WHERE NHL team = Vancouver Canucks

Test Instruction: When were the ships launched that were laid down on september 1, 1964?
Model Prediction: SELECT Date FROM table WHERE Launched = september 1, 1964
Expected Answer: SELECT Launched FROM table WHERE Laid down = September 1, 1964

Test Instruction: List the # for ships commissioned on september 30, 1967.
Model Prediction: SELECT # FROM table WHERE Ships commissioned FROM table WHERE Ships commissioned = september 30, 1967
Expected Answer: SELECT # FROM ta